## 分类数据

In [1]:
import numpy as np
import pandas as pd
#用unique和value_counts方法提取不同值和计数
values = pd.Series(['apple','orange','apple','apple']*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [2]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [3]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [4]:
#用重复值表示数据，比如维度表包含不同值，并将主要观测值存储为引用维度表的整数键
values = pd.Series([0,1,0,0]*2)
print(values)
dim = pd.Series(['apple','orange'])
#使用take方法恢复原来的字符串
dim.take(values)

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64


0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### Categorical类型

In [5]:
#Categorical类型，用于承载基于整数的类别展示或编码数据
fruits = ['apple','orange','apple','apple']*2
N = len(fruits)
df = pd.DataFrame({'fruit':fruits,'count':np.random.randint(3,15,size=N),
                   'weight':np.random.uniform(0,4,size=N),'basket_id':np.arange(N)},
                  columns=['basket_id','fruit','count','weight'])
df

,basket_id,fruit,count,weight
0,0,apple,7,1.635043
1,1,orange,5,2.876408
2,2,apple,8,3.046350
3,3,apple,8,2.157308
4,4,apple,12,3.098170
5,5,orange,4,3.632556
6,6,apple,3,0.234125
7,7,apple,5,0.441792


In [6]:
#将df['fruit']转化为Categorical类型
print(type(df['fruit'].values))
fruit_cat = df['fruit'].astype('category')
type(fruit_cat.values)


<class 'numpy.ndarray'>


pandas.core.arrays.categorical.Categorical

In [7]:
#Categorical有categories和codes两个属性
fruit_cat.values.categories

Index(['apple', 'orange'], dtype='object')

In [8]:
fruit_cat.values.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [9]:
#直接生成Categorical类型数据
my_categories = pd.Categorical(['foo','bar','baz','foo','bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [10]:
#如果已经有了分类编码数据，可以使用from_codes构造函数
categories = ['foo','bar','baz']
codes = [0,1,2,0,0,1]
my_cats = pd.Categorical.from_codes(codes,categories)
my_cats

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [11]:
#使用ordered参数指定顺序
ordered_cat = pd.Categorical.from_codes(codes,categories,ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [12]:
#使用Categorical对象进行计算
np.random.seed(12345)
draws = np.random.randn(1000)
#计算draws的四分位分箱
bins = pd.qcut(draws,4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [13]:
#把四分位数换成四分位数名称
bins = pd.qcut(draws,4,labels=['Q1','Q2','Q3','Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [14]:
bins.value_counts()

Q1    250
Q2    250
Q3    250
Q4    250
dtype: int64

In [15]:
#使用groupby获取统计值
bins = pd.Series(bins,name='quartile')
result = pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index()
result

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [16]:
result['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [17]:
#创建虚拟变量one-hot编码
cat_s = pd.Series(['a','b','c','d']*2,dtype='category')
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 高阶GroupBy应用

In [21]:
df = pd.DataFrame({'key':['a','b','c']*4,'value':np.arange(12)})
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [30]:
#在transfrom中传递所要应用的方法
g.transform(lambda x:x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [31]:
#或者直接传递字符串别名
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [32]:
g.transform(lambda x:x**2)

0       0
1       1
2       4
3       9
4      16
5      25
6      36
7      49
8      64
9      81
10    100
11    121
Name: value, dtype: int32

In [33]:
#按每个组的降序计算排名
g.transform(lambda x:x.rank(ascending=False))

0     4
1     4
2     4
3     3
4     3
5     3
6     2
7     2
8     2
9     1
10    1
11    1
Name: value, dtype: int32

In [42]:
#Z分数化
def normalize(x):
    return (x-x.mean())/x.std()
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [43]:
#apply
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### 分组的时间重新采样

In [49]:
N = 15
times = pd.date_range('2019-05-21 12:00:00',freq='1min',periods=N)
df = pd.DataFrame({'time':times,'value':np.arange(N)})
df

,time,value
0,2019-05-21 12:00:00,0
1,2019-05-21 12:01:00,1
2,2019-05-21 12:02:00,2
3,2019-05-21 12:03:00,3
4,2019-05-21 12:04:00,4
5,2019-05-21 12:05:00,5
6,2019-05-21 12:06:00,6
7,2019-05-21 12:07:00,7
8,2019-05-21 12:08:00,8
9,2019-05-21 12:09:00,9


In [54]:
#按time索引再重新采样
df.set_index('time').resample('5min').count()

,value
time,
2019-05-21 12:00:00,5
2019-05-21 12:05:00,5
2019-05-21 12:10:00,5


### 方法链技术

In [64]:
#使用assign进行赋值
df = pd.DataFrame({'key':['a','b','c','d']*2,'value':np.arange(8)})
#修改df['value']列
df['value'] = np.random.randint(0,10)
#用assign方法
df = df.assign(value=np.random.randint(0,10))

In [65]:
df

,key,value
0,a,9
1,b,9
2,c,9
3,d,9
4,a,9
5,b,9
6,c,9
7,d,9
